# **PROYECTO**

## COSMIC DEFENDER

In [7]:
import pygame #Se importa la librería pygame con todas sus funciones.
import random #Se importa la librería random para generación de eventos aleatorios.
import time #Se importa la librería time para generación de eventos dependientes del tiempo.

In [8]:
pygame.init() #Se inicia el constructor de pygame
pygame.mixer.init() #Se inicia el constructor de sonido de pygame.

## Configuraciones pantalla de inicio

In [9]:
anchoPantalla=800 #Se crea una variable con el ancho de pantalla.
altoPantalla=600 #Se crea una variable con el alto de pantalla.

#Se crea la pantalla del juego.
pantalla=pygame.display.set_mode((anchoPantalla,altoPantalla))
pygame.display.set_caption("Cosmic Defenders")

#Se ingresa la imagen de inicio en la pantalla.
pantallaDeInicio=pygame.image.load("Inicio-PrimerNivel.png").convert()
pantallaDeInicio=pygame.transform.scale(pantallaDeInicio,(800,600))

#Se ingresa el nombre del juego para mostrar en pantalla.
fuenteTitulo=pygame.font.Font("Silkscreen-Bold.ttf",50)
textoTitulo=fuenteTitulo.render("Cosmic Defenders",True,(0,0,0))
textoRect1=textoTitulo.get_rect(center=(anchoPantalla // 2,altoPantalla // 6))

#Se crea el botón de inicio del juego.
fuenteBotones=pygame.font.Font("Silkscreen-Bold.ttf",40)
textoInicio=fuenteBotones.render("INICIO",True,(255,255,255))
textoRect2=textoInicio.get_rect(center=(anchoPantalla // 2,altoPantalla // 1.5))
inicio=pygame.Rect(textoRect2.left - 1,textoRect2.top - 1,textoRect2.width + 1,textoRect2.height + 1)

#Se crea el botón de ajustes del juego.
textoAjustes=fuenteBotones.render("AJUSTES",True,(255,255,255))
textoRect3=textoAjustes.get_rect(center=(anchoPantalla // 2,altoPantalla // 1.3))
ajustes=pygame.Rect(textoRect3.left - 1,textoRect3.top - 1,textoRect3.width + 1,textoRect3.height + 1)

#Se crea la variable con el sonido de fondo.
musicaInicio=pygame.mixer.music.load("Himno Universidad de Santiago De Chile.mp3")
reproducirMusica=pygame.mixer.music.play(-1)

#Se crea la variable que contiene el sonido del disparo de la nave
sonidoDisparo=pygame.mixer.Sound("piun.mp3")
sonidoDisparoJefe=pygame.mixer.Sound("piuJefe.mp3")
sonidoJefeActivo=True

#Se crea un diccionario con la música para los niveles.
musicaNiveles={
    1: "primer_nivel.mp3",
    2: "segundo_nivel.mp3",
    3: "tercer_nivel.mp3",
    4: "ultimo_nivel.mp3"
}

## Clases

In [10]:
#Se crea la clase Jugador.
class Jugador(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.imagen=pygame.image.load("Nave.png").convert_alpha()
        self.imagen=pygame.transform.scale(self.imagen,(50,50))
        self.rect=self.imagen.get_rect()
        self.rect.center=(anchoPantalla // 2,altoPantalla-50)
        self.velocidad=10
        self.proyectiles=pygame.sprite.Group()
        self.enemigosDerribados=0
        self.vida=5

    def mover(self,teclas):
        #Función para mover la nave según se indica el jugador (izquierda o derecha).
        if teclas[pygame.K_LEFT] and self.rect.left > 0:
            self.rect.x-=self.velocidad
            
        if teclas[pygame.K_RIGHT] and self.rect.right < anchoPantalla:
            self.rect.x+=self.velocidad

    def aparicion(self,pantalla):
        #Función para mostrar la nave en pantalla.
        pantalla.blit(self.imagen,self.rect)

    def disparar(self):
        #Función para disparar los proyectiles.
        proyectil=Proyectil(self.rect.centerx,self.rect.top)
        self.proyectiles.add(proyectil)

    def recibirDaño(self):
        #Función para que la nave reciba daño por chocar con meteoritos o recibir disparos del jefe final.
        self.vida-=1
        
    def actualizar(self,meteoritos,puntaje):
        #Función para actualizar los distintos objetos en pantalla y eliminar objetos si es necesario.

        #Se muestran los proyectiles en pantalla.
        for proyectil in self.proyectiles:
            proyectil.mover()
            proyectil.aparecer(pantalla)

            #Se recdorre la lista de meteoritos.
            for meteorito in meteoritos:
                
                #Condicional para verificar cuando se tocan los proyectiles y los meteoritos, se eliminan en el caso que corresponda.
                if proyectil.rect.colliderect(meteorito.rect):
                    puntosObtenidos=meteorito.recibirDisparo()
                    puntaje+=puntosObtenidos
                    proyectil.kill()
                    self.enemigosDerribados+=1
                    
        #Se recdorre la lista de meteoritos.            
        for meteorito in meteoritos:

            #Condicional para verificar que los meteoritos tocan a la nave, si esto ocurre se elimina el meteorito y se resta vida de la nave.
            if self.rect.colliderect(meteorito.rect):
                self.recibirDaño()
                meteorito.kill() 
                
        return puntaje
        
    def actualizarJefe(self,puntaje,jefe,jefeActivo):
        #Función para actualizar los distintos objetos en pantalla y eliminar objetos si es necesario, en este caso el jefe igual cuenta.

        #Se muestran los proyectiles en pantalla.
        for proyectil in self.proyectiles:
            proyectil.mover()
            proyectil.aparecer(pantalla)

            #Condicional para detectar al jefe en pantalla y que este reciba daño.
            if isinstance(jefe,JefeFinal) and jefeActivo == True and proyectil.rect.colliderect(jefe.rect):
                puntosObtenidos=jefe.recibirDisparo()
                puntaje+=puntosObtenidos
                proyectil.kill()
                
        #Se recorre la lista de disparos del jefe final.       
        for proyectilJefe in jefe.proyectiles:

            #Condicional para verificar que el proyectil del jefe toca la nave y realiza daño.
            if self.rect.colliderect(proyectilJefe.rect):
                self.recibirDaño()
                proyectilJefe.kill()
                
        return puntaje

#Se crea la clase Proyectil.
class Proyectil(pygame.sprite.Sprite):
    def __init__(self,x,y):
        super().__init__()
        self.imagen=pygame.image.load("proyectil.png").convert_alpha()
        self.imagen=pygame.transform.scale(self.imagen,(10,30))
        self.rect=self.imagen.get_rect()
        self.rect.centerx=x
        self.rect.bottom=y
        self.velocidad=7

    def mover(self):
        #Función para disparar los proyectiles de la nave verticalmente hacia arriba.
        self.rect.y-=self.velocidad
        
        if self.rect.bottom < 0:
            self.kill()

    def aparecer(self,pantalla):
        #Función para mostrar los proyectiles de la nave en pantalla.
        pantalla.blit(self.imagen,self.rect)

#Se crea la clase madre Enemigos.        
class Enemigos(pygame.sprite.Sprite):
    def __init__(self,enemigos,velocidad,tamano,disparosNecesarios,puntaje):
        super().__init__()
        self.imagen=pygame.image.load(enemigos).convert_alpha()
        self.imagen=pygame.transform.scale(self.imagen,tamano)
        self.rect=self.imagen.get_rect()
        self.velocidad=velocidad
        self.disparosRecibidos=0
        self.disparosNecesarios=disparosNecesarios 
        self.puntaje=puntaje

    def caer(self):
        #Función para que los meteoritos caigan verticalmente hacia abajo.
        self.rect.y+=self.velocidad

    def mostrar(self,pantalla):
        #Función para mostrar los meteoritos en pantalla.
        pantalla.blit(self.imagen,self.rect)

    def recibirDisparo(self):
        #Función para que los meteoritos reciban daño y se destruyan.
        self.disparosRecibidos+=1
        if self.disparosRecibidos >= self.disparosNecesarios:
            self.kill()
            
            return self.puntaje
            
        return 0

#Se crea la clase hija Meteorito1.
class Meteorito1(Enemigos):
    def __init__(self):
        super().__init__("meteorito1.png",velocidad=1,tamano=(50,70),disparosNecesarios=1,puntaje=10)
        x=random.randint(0,anchoPantalla-70)
        y=0
        self.rect=pygame.Rect(x,y,50,70)

#Se crea la clase hija Meteorito2.
class Meteorito2(Enemigos):
    def __init__(self):
        super().__init__("meteorito2.png",velocidad=2,tamano=(50,70),disparosNecesarios=2,puntaje=20)
        x=random.randint(0,anchoPantalla-70)
        y=0
        self.rect=pygame.Rect(x,y,50,70)

#Se crea la clase hija Meteorito3.
class Meteorito3(Enemigos):
    def __init__(self):
        super().__init__("meteorito3.png",velocidad=1,tamano=(120,150),disparosNecesarios=4,puntaje=40)
        x=random.randint(0,anchoPantalla-150)
        y=0
        self.rect=pygame.Rect(x,y,120,150)    

#Se crea la calse JefeFinal.      
class JefeFinal(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.imagen=pygame.image.load("jefe_final.png").convert_alpha()
        self.imagen=pygame.transform.scale(self.imagen,(200,200))
        self.rect=self.imagen.get_rect()
        self.rect.center=(anchoPantalla // 2,100)
        self.velocidad=3
        self.direccion=1
        self.disparosNecesarios=10
        self.disparosRecibidos=0
        self.puntaje=500
        self.proyectiles=pygame.sprite.Group()
        self.tiempoUltimoDisparo=0

    def mover(self):
        #Función para que el jefe final se mueva horizontalmente.
        self.rect.x+=self.velocidad
        
        #Condicional para que el jefe se desplaze hacia la derecha.
        if self.rect.right > anchoPantalla:
            self.rect.right=anchoPantalla
            self.velocidad*=-1

        #Condicional para que el jefe se desplaze hacia la izquierda.
        if self.rect.left < 0:
            self.rect.left=0
            self.velocidad*=-1

    def recibirDisparo(self):
        #Función para que el jefe reciba daño.
        self.disparosRecibidos+=1
        if self.disparosRecibidos >= self.disparosNecesarios:
            self.kill()
            
            return self.puntaje
            
        return 0
    
    def disparar(self):
        #Función para que el jefe realize los disparos.
        if pygame.time.get_ticks()-self.tiempoUltimoDisparo > 500 and random.randrange(0,1000,1) <= 50:  # Dispara cada segundo
            proyectilJefeIzquierda=ProyectilJefe(self.rect.centerx-30,self.rect.bottom)
            proyectilJefeDerecha=ProyectilJefe(self.rect.centerx+30,self.rect.bottom)
            
            if sonidoJefeActivo:
                sonidoDisparoJefe.play()
                
            self.proyectiles.add(proyectilJefeIzquierda)
            self.proyectiles.add(proyectilJefeDerecha)
            self.tiempoUltimoDisparo=pygame.time.get_ticks()
            
    def mostrar(self,pantalla):
        #Función para mostrar al jefe final en pantalla.
        pantalla.blit(self.imagen,self.rect)
        
    def actualizarProyectiles(self,pantalla):
        #Función para actualizar y mostrar los proyectiles disparados por el jefe.
        for proyectil in self.proyectiles:
            proyectil.mover()
            proyectil.aparecer(pantalla)

#Se crea la clase ProyectilJefe.
class ProyectilJefe(pygame.sprite.Sprite):
    def __init__(self,x,y):
        super().__init__()
        self.imagen=pygame.image.load("proyectil_jefe.png").convert_alpha()
        self.imagen=pygame.transform.scale(self.imagen, (20, 40))
        self.rect=self.imagen.get_rect()
        self.rect.centerx=x
        self.rect.top=y
        self.velocidad=5

    def mover(self):
        #Función para que los pryectiles se muevan verticalmente hacia abajo.
        self.rect.y+=self.velocidad

        if self.rect.top > altoPantalla:
            self.kill()

    def aparecer(self,pantalla):
        #Función para mostrar los proyectiles en pantalla.
        pantalla.blit(self.imagen,self.rect)

## Funciones

In [11]:
def pantallaAjustes():
    """
    Función para definir la pantalla de ajustes.
    -------------------------------------------------------------------
    Retorna:
        textoRectSalir = Variable para salir de la pantalla de ajustes.
        textoRectOnMusica = Variable para encender la música.
        textoRectOffMusica = Variable para apagar la música.
        textoRectOnSonido = Variable para encender el sonido del juego.
        textoRectOffSonido = Variable para apagar el sonido del juego.
    """
    #Se muestra la pantalla negra para presentar los ajustes.
    pantalla.fill((0,0,0))

    #Se crea el botón de ajustar musica del juego.
    fuenteAjustes=pygame.font.Font("Silkscreen-Bold.ttf",50)
    textoMusica=fuenteAjustes.render("Musica",True,(255,255,255))
    textoRectMusica=textoMusica.get_rect(center=(anchoPantalla // 2,altoPantalla // 5))

    #Se crea el botón de encender la música.
    fuenteOnMusica=pygame.font.Font("Silkscreen-Bold.ttf",30)
    textoOnMusica=fuenteOnMusica.render("ON",True,(255,255,255))
    textoRectOnMusica=textoOnMusica.get_rect(center=(anchoPantalla // 2.3,altoPantalla // 3))
    pygame.draw.rect(pantalla,(0,255,0),textoRectOnMusica)

    #Se crea el botón de apagar la música.
    fuenteOffMusica=pygame.font.Font("Silkscreen-Bold.ttf",30)
    textoOffMusica=fuenteOffMusica.render("OFF",True,(255,255,255))
    textoRectOffMusica=textoOffMusica.get_rect(center=(anchoPantalla // 1.7,altoPantalla // 3))
    pygame.draw.rect(pantalla,(255,0,0),textoRectOffMusica)

    #Se crea el botón de ajustar sonido del juego.
    fuenteSonido=pygame.font.Font("Silkscreen-Bold.ttf",50)
    textoSonido=fuenteSonido.render("Sonidos",True,(255,255,255))
    textoRectSonido=textoSonido.get_rect(center=(anchoPantalla // 2,altoPantalla // 2))

    #Se crea el botón de encender el sonido.
    fuenteOnSonido=pygame.font.Font("Silkscreen-Bold.ttf",30)
    textoOnSonido=fuenteOnSonido.render("ON",True,(255,255,255))
    textoRectOnSonido=textoOnSonido.get_rect(center=(anchoPantalla // 2.3,altoPantalla // 1.6))
    pygame.draw.rect(pantalla,(0,255,0),textoRectOnSonido)

    #Se crea el botón de apagar el sonido.
    fuenteOffSonido=pygame.font.Font("Silkscreen-Bold.ttf",30)
    textoOffSonido=fuenteOffSonido.render("OFF",True,(255,255,255))
    textoRectOffSonido=textoOffSonido.get_rect(center=(anchoPantalla // 1.7,altoPantalla // 1.6))
    pygame.draw.rect(pantalla,(255,0,0),textoRectOffSonido)

    #Se crea el botón de salir de los ajustes.
    fuenteSalir=pygame.font.Font("Silkscreen-Bold.ttf",50)
    textoSalir=fuenteSalir.render("Salir",True,(255,255,255))
    textoRectSalir=textoSalir.get_rect(center=(anchoPantalla // 2,altoPantalla // 1.3))

    #Se muestran los botones en pantalla.
    pantalla.blit(textoMusica,textoRectMusica)
    pantalla.blit(textoOnMusica,textoRectOnMusica)
    pantalla.blit(textoOffMusica,textoRectOffMusica)
    pantalla.blit(textoSonido,textoRectSonido)
    pantalla.blit(textoOnSonido,textoRectOnSonido)
    pantalla.blit(textoOffSonido,textoRectOffSonido)
    pantalla.blit(textoSalir,textoRectSalir)

    return textoRectSalir,textoRectOnMusica,textoRectOffMusica,textoRectOnSonido,textoRectOffSonido

def pantallaPerdida():
    """
    Función para definir la pantalla de perdida.
    -------------------------------------------------------------------
    Retorna:
        textoRectSalir = Variable para salir de la pantalla de ajustes.
        textoRectOnMusica = Variable para encender la música.
        textoRectOffMusica = Variable para apagar la música.
        textoRectOnSonido = Variable para encender el sonido del juego.
        textoRectOffSonido = Variable para apagar el sonido del juego.
    """
    #Se muestra la pantalla negra para presentar los ajustes.
    pantalla.fill((0,0,0))

    #Se crea el botón de ajustar musica del juego.
    fuenteDerrota=pygame.font.Font("Silkscreen-Bold.ttf",50)
    textoDerrota=fuenteDerrota.render("Has perdido",True,(255,255,255))
    textoRectDerrota=textoDerrota.get_rect(center=(anchoPantalla // 2,altoPantalla // 5))
    
    #Se crea el botón de salir de los ajustes.
    fuenteSalir=pygame.font.Font("Silkscreen-Bold.ttf",50)
    textoSalir=fuenteSalir.render("Salir",True,(255,255,255))
    textoRectSalir=textoSalir.get_rect(center=(anchoPantalla // 2,altoPantalla // 1.2))

    #Se crea el botón de salir de los ajustes.
    fuenteReiniciar=pygame.font.Font("Silkscreen-Bold.ttf",50)
    textoReiniciar=fuenteReiniciar.render("Rieniciar",True,(255,255,255))
    textoRectReiniciar=textoReiniciar.get_rect(center=(anchoPantalla // 2,altoPantalla // 1.5))

    #Se muestran los botones en pantalla.
    pantalla.blit(textoDerrota,textoRectDerrota)
    pantalla.blit(textoSalir,textoRectSalir)
    pantalla.blit(textoReiniciar,textoRectReiniciar)

    pygame.display.update()

    return textoRectReiniciar,textoRectSalir
    
def mostrarPuntaje(pantalla,puntaje):
    """
    Función para mostrar el puntaje en pantalla.
    ----------------------------------------------------------------
    Recibe:
    pantalla = Varible que representa la pantalla del juego. 
    puntaje = Variable que representa el puntaje actual del jugador.
    """
    fuentePuntaje=pygame.font.Font("Silkscreen-Bold.ttf",22)
    textoPuntaje=fuentePuntaje.render(f"Puntuación: {puntaje}",True,(255,255,255))
    pantalla.blit(textoPuntaje,(anchoPantalla-300,10))
    
def reproducirMusica(nivel):
    """
    Función para cargar y reproducir música de nivel.
    """
    if nivel in musicaNiveles:
        pygame.mixer.music.stop()
        pygame.mixer.music.load(musicaNiveles[nivel])
        pygame.mixer.music.play(-1)

#Se crean las variables necesarias para manejar los enemigos del juego.
niveles=1  
meteoritos=[]  
crearMeteoritos=0  
tiempoDeCreacion=5500  
cantidadDeMeteoritos=2000  
contadorDeMeteoritos=0  
jefeActivo=False

def enemigosPorNivel(niveles,meteoritos,crearMeteoritos,tiempoDeCreacion,cantidadDeMeteoritos,contadorDeMeteoritos,jugador,nivelCambiado,jefe,jefeActivo):
    """
    Función para generar enemigos por cada nivel del juego.
    ----------------------------------------------------------------------------------
    Recibe:
        niveles = Nivel actual.
        meteoritos = Lista para almacenar los meteoritos.
        crearMeteoritos = Controlador del tiempo para crear meteoritos.
        tiempoDeCreacion = Intervalo en milisegundos para crear meteoritos.
        cantidadDeMeteoritos = Número total de meteoritos por nivel.
        contadorDeMeteoritos = Contador para saber cuántos meteoritos han sido generados.
        jugador = Instancia de jugador.

    Retorna:
        meteoritos = Lista de meteoritos.
        crearMeteoritos = Entero que contiene el tiempo de creación de los meteoritos. 
        contadorDeMeteoritos = Entero que contiene la cantidad de meteoritos creados.
        niveles = Entero que contiene el nivel del juego.
        nivelCambiado = Entero que contiene el cambio de nivel.
    """
    #Controlamos el tiempo entre meteoritos.
    tiempoNivel=pygame.time.get_ticks()
    #Condicionales para crear meteoritos por cada nivel.
    if (tiempoNivel-crearMeteoritos >= tiempoDeCreacion) and (contadorDeMeteoritos < cantidadDeMeteoritos):
        #Condicional con la aparición específica de los meteoritos para el nivel 1.
        if niveles == 1:
            filaDeMeteoritos=5
            for i in range(filaDeMeteoritos):
                meteoritos.add(Meteorito1())
        
            if jugador.enemigosDerribados >= 5:
                niveles+=1
                jugador.enemigosDerribados=0
                nivelCambiado=True
                meteoritos.empty()

        #Condicional con la aparición específica de los meteoritos para el nivel 2.
        elif niveles == 2:
            filaDeMeteoritos=6
            for i in range(filaDeMeteoritos):
                meteoritos.add(Meteorito1())  

            if jugador.enemigosDerribados >= 5: 
                niveles+=1  
                jugador.enemigosDerribados=0
                nivelCambiado=True
                meteoritos.empty()

        #Condicional con la aparición específica de los meteoritos para el nivel 3.
        elif niveles == 3:
            filaDeMeteoritos=4
            for i in range(filaDeMeteoritos):
                meteoritos.add(Meteorito1())  

            if jugador.enemigosDerribados >= 5:  
                niveles+=1 
                jugador.enemigosDerribados=0
                nivelCambiado=True
                meteoritos.empty()

        #Condicional con la aparición específica de los meteoritos para el nivel 4.
        elif niveles == 4:
            filaDeMeteoritos=4
            for i in range(filaDeMeteoritos):
                numeroRandom=random.randrange(0,100,1)
                if numeroRandom < 30:
                    meteoritos.add(Meteorito2())  
                else:
                    meteoritos.add(Meteorito1())
                    
            if jugador.enemigosDerribados >= 5:  
                niveles+=1  
                jugador.enemigosDerribados=0
                nivelCambiado=True
                meteoritos.empty()

        #Condicional con la aparición específica de los meteoritos para el nivel 5.
        elif niveles == 5:
            filaDeMeteoritos=4
            for i in range(filaDeMeteoritos):
                numeroRandom=random.randrange(0,100,1)
                if numeroRandom < 50:
                    meteoritos.add(Meteorito2())  
                else:
                    meteoritos.add(Meteorito1())
                    
            if jugador.enemigosDerribados >= 5:  
                niveles+=1 
                jugador.enemigosDerribados=0
                nivelCambiado=True
                meteoritos.empty()

        #Condicional con la aparición específica de los meteoritos para el nivel 6.
        elif niveles == 6:
            filaDeMeteoritos=4
            for i in range(filaDeMeteoritos):
                numeroRandom=random.randrange(0,100,1)
                if numeroRandom < 60:
                    meteoritos.add(Meteorito2())  
                else:
                    meteoritos.add(Meteorito1())
                    
            if jugador.enemigosDerribados >= 5:  
                niveles+=1  
                jugador.enemigosDerribados=0
                nivelCambiado=True
                meteoritos.empty()

        #Condicional con la aparición específica de los meteoritos para el nivel 7.
        elif niveles == 7:
            filaDeMeteoritos=4
            for i in range(filaDeMeteoritos):
                numeroRandom=random.randrange(0,100,1)
                numeroRandom2=random.randrange(0,100,1)
                if numeroRandom < 60:
                    meteoritos.add(Meteorito2())  
                if numeroRandom2 < 30:
                    meteoritos.add(Meteorito3())
                else:
                    meteoritos.add(Meteorito1())
                    
            if jugador.enemigosDerribados >= 5:  
                niveles+=1  
                jugador.enemigosDerribados=0
                nivelCambiado=True
                meteoritos.empty()

        #Condicional con la aparición específica de los meteoritos para el nivel 8.
        elif niveles == 8:
            filaDeMeteoritos=4
            for i in range(filaDeMeteoritos):
                numeroRandom=random.randrange(0,100,1)
                numeroRandom2=random.randrange(0,100,1)
                if numeroRandom < 60:
                    meteoritos.add(Meteorito2())  
                if numeroRandom2 < 40:
                    meteoritos.add(Meteorito3())
                else:
                    meteoritos.add(Meteorito1())
                    
            if jugador.enemigosDerribados >= 5:  
                niveles+=1  
                jugador.enemigosDerribados=0
                nivelCambiado=True
                meteoritos.empty()

        #Condicional con la aparición específica de los meteoritos para el nivel 9.
        elif niveles == 9:
            filaDeMeteoritos=4
            for i in range(filaDeMeteoritos):
                numeroRandom=random.randrange(0,100,1)
                numeroRandom2=random.randrange(0,100,1)
                if numeroRandom < 60:
                    meteoritos.add(Meteorito2())  
                if numeroRandom2 < 50:
                    meteoritos.add(Meteorito3())
                else:
                    meteoritos.add(Meteorito1())
                    
            if jugador.enemigosDerribados >= 10:  
                niveles+=1  
                jugador.enemigosDerribados=0
                nivelCambiado=True
                meteoritos.empty()

                
        elif niveles == 10:  # Si llegamos al nivel 10, agregamos el jefe final
            filaDeMeteoritos = 4
            for i in range(filaDeMeteoritos):
                numeroRandom=random.randrange(0,100,1)
                numeroRandom2=random.randrange(0,100,1)
                if numeroRandom < 50:
                    meteoritos.add(Meteorito2())  
                if numeroRandom2 < 50:
                    meteoritos.add(Meteorito3())
                else:
                    meteoritos.add(Meteorito1())
            if jefeActivo==False: # Se verifica que no este el jefe
                meteoritos.add(jefe)
                jefeActivo=True 


        contadorDeMeteoritos+=filaDeMeteoritos #Se actualiza el valor del contador de meteoritos.
        crearMeteoritos=tiempoNivel #Se actualiza el tiempo de creación de los meteoritos.

    return meteoritos,crearMeteoritos,contadorDeMeteoritos,niveles,nivelCambiado,jefeActivo

def mostrarMensaje(pantalla,mensaje,color,pos_x,pos_y,font_size=36):
    """
    Función para mostrar un mensaje en la pantalla.
    """
    pantalla.fill((0,0,0))
    fuente=pygame.font.Font("Silkscreen-Bold.ttf",font_size)
    texto=fuente.render(mensaje,True,color)
    rect=texto.get_rect(center=(pos_x,pos_y))
    pantalla.blit(texto,rect)
    pygame.display.flip()

def reproducirMusica(nivel):
    """
    Reproduce la música correspondiente al nivel.
    """
    pygame.mixer.music.stop() #Detener cualquier música en reproducción.

    #Condicionales para cambiar la música según el nivel.
    if nivel >= 1 and nivel < 4:
        archivoMusica="primer_nivel.mp3"
            
    elif nivel >= 4 and nivel < 7:
        archivoMusica="segundo_nivel.mp3"
            
    elif nivel >= 7 and nivel < 10:
        archivoMusica="tercer_nivel.mp3"
            
    else:
        archivoMusica = "ultimo_nivel.mp3"
            
    pygame.mixer.music.load(archivoMusica)
    pygame.mixer.music.play(-1)  #Reproducir en bucle la música.

def cambiarPantallaNiveles(nivel):
    """
    Función para cambiar el fondo según el nivel.
    """
    #Se crean las variables que contienen las imagenés con los fondos de los niveles.
    pantallaNiveles1=pygame.image.load("Inicio-PrimerNivel.png")
    pantallaNiveles2= pygame.image.load("Niveles(4-6).png")
    pantallaNiveles3=pygame.image.load("Niveles(7-9).png")
    pantallaNivelFinal=pygame.image.load("NivelFinal.png")

    pantallaNiveles1=pygame.transform.scale(pantallaNiveles1,(anchoPantalla,altoPantalla))
    pantallaNiveles2=pygame.transform.scale(pantallaNiveles2,(anchoPantalla,altoPantalla))
    pantallaNiveles3=pygame.transform.scale(pantallaNiveles3,(anchoPantalla,altoPantalla))
    pantallaNivelFinal=pygame.transform.scale(pantallaNivelFinal,(anchoPantalla,altoPantalla))

    #Condicionales para cambiar el fondo según el nivel.
    if nivel >= 1 and nivel < 4:
        pantalla.blit(pantallaNiveles1,(0,0))
    
    elif nivel >= 4 and nivel < 7:
        pantalla.blit(pantallaNiveles2,(0,0))
            
    elif nivel >= 7 and nivel < 10:
        pantalla.blit(pantallaNiveles3,(0,0))
            
    else:
        pantalla.blit(pantallaNivelFinal,(0,0))

def reproducirMusicaMenu():
    """
    Reproduce la música del menú principal.
    """
    pygame.mixer.music.load("Himno Universidad de Santiago De Chile.mp3")
    pygame.mixer.music.play(-1)

## Ejecucion

In [12]:
#Se hace la configuración inicial.
clock=pygame.time.Clock()
nave=Jugador()
meteoritos=pygame.sprite.Group()
puntaje=0
ultimoTiempo=time.time()
jefe=JefeFinal()

#Se crean las variables booleanas para utilizar durante la ejecución.
ejecucion=True
ingresarAjustes=False
inicioJuego=False
escucha=True
musica=True
opcionesDerrota=True

#Se crean las variables para el manejo de los niveles del juego.
nivelActual=1
enemigosDerribados=0
nivelCambiado=False
tiempoEsperaNivel=2000
ultimoTiempoNivel=time.time()

#Se hace un ciclo para ejecutar el juego.
while ejecucion:
    pantalla.fill((0,0,0))
    #Se crea un ciclo con el apartado de opciones de la pantalla de inicio del juego.
    for opcionesJuego in pygame.event.get():
        if opcionesJuego.type == pygame.QUIT: #Si se presiona la salida de la pestaña de juego, se termina la ejecución.
            ejecucion=False

        if opcionesJuego.type == pygame.MOUSEBUTTONDOWN:
            if ajustes.collidepoint(opcionesJuego.pos): #Si se hace clic en el botón de ajustes, se ingresa a la pantalla de ajustes.
                ingresarAjustes=True

            if inicio.collidepoint(opcionesJuego.pos) and ingresarAjustes == False: #Si se hace clic en el botón de inicio, comienza el juego.
                inicioJuego=True
                if musica == True:
                    reproducirMusica(nivelActual) #Se reproduce la música según el nivel en que se encuentre.
                else:
                    pygame.mixer.music.stop()

        if opcionesJuego.type == pygame.KEYDOWN:
            if opcionesJuego.key == pygame.K_SPACE: #Si se presiona la tecla de disparo, la nave dispara proyectiles.
                nave.disparar()
                if escucha == True: #Si la nave dispara, se efectua el sonido del disparo.
                    sonidoDisparo.play(loops=0)
                else:
                    sonidoDisparo.stop()
    #Se crea un condicional con el apartado de opciones de la pantalla de juego en los niveles.
    if inicioJuego:
        cambiarPantallaNiveles(nivelActual) #Se actualiza el fondo de los niveles según corresponda.
        teclas=pygame.key.get_pressed() #Se detectan las teclas presionadas.
        nave.mover(teclas) #Movemos al jugador según las teclas presionadas.
        nave.aparicion(pantalla) #Mostramos la pantalla del primer nivel.
        #Se llama a la función para generar enemigos para cada nivel.
        meteoritos,crearMeteoritos,contadorDeMeteoritos,nivelActual,nivelCambiado,jefeActivo=enemigosPorNivel(nivelActual,\
                                                                                                                   meteoritos,crearMeteoritos,\
                                                                                                   tiempoDeCreacion,cantidadDeMeteoritos,\
                                                                                                   contadorDeMeteoritos,nave,nivelCambiado,\
                                                                                                   jefe,jefeActivo)

        for enemigo in meteoritos:
            # Si el enemigo es un meteorito
            if isinstance(enemigo,Enemigos):
                enemigo.caer()  # Solo meteoritos tienen este método
            elif isinstance(enemigo,JefeFinal):
                # Si es el jefe, aplicamos sus métodos especiales
                puntaje=nave.actualizarJefe(puntaje,enemigo,jefeActivo)
                enemigo.mover()  # Movimiento del jefe
                enemigo.disparar()  # El jefe dispara
                enemigo.actualizarProyectiles(pantalla)  # Gestiona los proyectiles del jefe
        
            enemigo.mostrar(pantalla)  #Todos los enemigos se dibujan

        #Se configura la puntuación.
        puntaje=nave.actualizar(meteoritos,puntaje)
            
        if time.time()-ultimoTiempo >= 1: #Si ha pasado 1 segundo, se incrementa la puntuación.
            puntaje+=1
            ultimoTiempo=time.time() #Se actualiza el último tiempo de incremento de puntuación.

        #Se crea un condicional para realizar cambios por cada nivel superado.
        if nivelCambiado == True:
            #Se muestra una pantalla que indica el fin de un nivel y da paso al siguiente.
            mostrarMensaje(pantalla,f"Nivel {nivelActual-1} superado",(255,255,255),400,300,font_size=48)
            pygame.display.update()
            pygame.time.delay(tiempoEsperaNivel)

            #Cambiar música al nuevo nivel.
            if musica == True:
                reproducirMusica(nivelActual)
            nivelCambiado=False

        mostrarPuntaje(pantalla,puntaje) #Se muestra el puntaje en pantalla.
        pygame.display.update() #Se actualiza la pantalla.

        if nave.vida <= 0:
            reiniciarNiveles,salirNiveles=pantallaPerdida()
            pygame.mixer.music.stop()
            pygame.display.update()

            opcionesDerrota=True
            while opcionesDerrota:
                for evento in pygame.event.get():
                    if evento.type == pygame.MOUSEBUTTONDOWN:
                        if salirNiveles.collidepoint(evento.pos):
                            inicioJuego=False
                            ingresarAjustes=False
                            puntaje=0
                            nave=Jugador()
                            meteoritos.empty()
                            
                            pygame.mixer.music.stop()
                            if musica:
                                reproducirMusicaMenu()
                            opcionesDerrota=False
                            
                        elif reiniciarNiveles.collidepoint(evento.pos):
                            nave=Jugador()
                            meteoritos.empty()
                            puntaje=0
                            inicioJuego=True
                            nivelActual=1
                            
                            pygame.mixer.music.stop()
                            if musica:
                                reproducirMusica(nivelActual)
                            opcionesDerrota=False

    #Se crea un condicional con el apartado de opciones de la pantalla de ajustes del juego.
    if ingresarAjustes:
        pantallaAjustes() #Mostramos la pantalla de ajustes.
        salirAjustes,onMusica,offMusica,onSonido,offSonido=pantallaAjustes()
        pygame.display.update()  #Se actualiza la pantalla de ajustes.

        for opcionesAjustes in pygame.event.get():
            if opcionesAjustes.type == pygame.MOUSEBUTTONDOWN:
                if offMusica.collidepoint(opcionesAjustes.pos):
                    pygame.mixer.music.stop()
                    musica=False #Si se apreta el botón on se escuchará la música.

                if onMusica.collidepoint(opcionesAjustes.pos):
                    if pygame.mixer.music.get_busy() == False:
                        pygame.mixer.music.play(-1)
                        musica=True #Si se apreta el botón off no se escuchará la música.

                if offSonido.collidepoint(opcionesAjustes.pos):
                    escucha=False #Si se apreta el botón off no se escuchará el sonido del juego.
                    sonidoJefeActivo=False

                if onSonido.collidepoint(opcionesAjustes.pos):
                    escucha=True #Si se apreta el botón on se escuchará el sonido del juego.

            if opcionesAjustes.type == pygame.QUIT: #Si se presiona la salida de la pestaña de ajustes, se termina la ejecución.
                ejecucion=False

            if opcionesAjustes.type == pygame.MOUSEBUTTONDOWN:
                if salirAjustes.collidepoint(opcionesAjustes.pos): #Si se presiona la salida de los ajustes, se vuelve a la pantalla de inicio.
                    ingresarAjustes=False
                    
    #Se crea un condicional para volver a la pantalla de inicio.
    if (ingresarAjustes == False) and (inicioJuego == False):
        pantalla.blit(pantallaDeInicio,[0,0])
        pantalla.blit(textoTitulo,textoRect1)

        pygame.draw.rect(pantalla,(160,32,240),inicio)
        pantalla.blit(textoInicio,textoRect2)

        pygame.draw.rect(pantalla,(160, 32, 240),ajustes)
        pantalla.blit(textoAjustes,textoRect3)

    pygame.display.flip() #Se actualiza la pantalla.
    clock.tick(60) #Se manejan los FPS.

pygame.quit()